In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


# BASIC SETUP

In [ ]:
# BASIC SETUP
! [ ! -z "$COLAB_GPU" ] && pip install torch skorch && pip install neptune-client

!cp "drive/My Drive/dl_project_data/repo/data_loading.py" .
!mkdir ./helper_scripts/
!cp "drive/My Drive/dl_project_data/repo/helper_scripts/visual_helpers.py" ./helper_scripts
!cp "drive/My Drive/dl_project_data/repo/architecture.py" .
!cp "drive/My Drive/dl_project_data/repo/model_training.py" .
!mkdir ./train/
# Creates RAM-Disk for potential speed-up
!sudo mount -t tmpfs -o size=7g tmpfs train
!for i in 0; do cp "drive/My Drive/dl_project_data/train/$i.tar" ./train/; tar -xf "./train/$i.tar" -C ./train/; rm "./train/$i.tar"; done;
#!for i in 0 1 2 3 4 5 6 7 8 9 10 11; do cp "drive/My Drive/dl_project_data/train/$i.tar" ./train/; tar -xf "./train/$i.tar" -C ./train/; rm "./train/$i.tar"; done;

     |████████████████████████████████| 122kB 8.0MB/s 
     |████████████████████████████████| 92kB 4.9MB/s 
     |████████████████████████████████| 829kB 15.9MB/s 
     |████████████████████████████████| 61kB 7.6MB/s 
     |████████████████████████████████| 204kB 18.0MB/s 
     |████████████████████████████████| 163kB 24.4MB/s 
     |████████████████████████████████| 143kB 28.2MB/s 
     |████████████████████████████████| 133kB 41.2MB/s 
     |████████████████████████████████| 71kB 7.8MB/s 
     |████████████████████████████████| 71kB 8.5MB/s 
  Created wheel for neptune-client: filename=neptune_client-0.4.119-py2.py3-none-any.whl size=150019 sha256=488665985e504465c0df302f98f200b850691c739b365975ba1b38992429fb91
  Stored in directory: /root/.cache/pip/wheels/84/5c/c1/a81e80761b94b4467fd3fda1fd3109463702f6247fc422eb33
  Created wheel for future: filename=future-0.18.2-cp36-none-any.whl size=491057 sha256=63b50c7e29bc58c2f820386c9f6b59b02041cdfadc5d24482ea8d4ecba79d79d
  Stored in dire

# IMPORTS
**You should not have to change anything here.**

In [ ]:
# IMPORTS
from collections import OrderedDict

import torch
from torch import nn
from torchvision import models, transforms

import skorch.callbacks as scb
from skorch import NeuralNetBinaryClassifier

import model_training as md
import architecture as arch
from data_loading import ToTensor, Normalize, RandomRotation, RandomHorizontalFlip

# Skorch uses some depricated scikit code - works just fine, so surpress warnings for clean output
import warnings
warnings.filterwarnings('ignore')

# CLASSIFIER PARAMETRIZATION

Here you can parametrize your model and set loss, optimizer, learning rate, etc. 

For further information on what can be set and how, please refer to the [skorch documentation](https://skorch.readthedocs.io/en/stable/classifier.html#skorch.classifier.NeuralNetClassifier).

In [ ]:
# CLASSIFIER PARAMETRIZATION
classifier = NeuralNetBinaryClassifier(
    arch.ResNet152,
    optimizer = torch.optim.Adam, 
    max_epochs = 2,
    lr = 0.01,
    batch_size = 64,
    iterator_train__shuffle = True, # Shuffle training data on each epoch
    train_split = None,
    callbacks = [scb.LRScheduler(policy = 'StepLR', gamma = 0.4, step_size = 2.0)], 
    device ='cuda')

# CLASSIFIER TRAINING

After you have added the shared folder with the data to your drive as a shortcut, you should not have to change anything here. At least for now.

**IF YOU WANT TO TRAIN WITH THE FULL DATASET, JUST REMOVE** *_small* **FROM THE CSV FILE.** 

In [ ]:
# CLASSIFIER TRAINING
md.train_model(classifier, 
            train_labels = "drive/My Drive/dl_project_data/train/train_split_small.csv", 
            test_labels = "drive/My Drive/dl_project_data/train/test_split_small.csv", 
            file_dir = "train", 
            train_transform = transforms.Compose([transforms.ToPILImage(),
                                  transforms.Pad(64, padding_mode='reflect'), # 96 + 2*64 = 224
                                  transforms.RandomHorizontalFlip(),  # TODO: model expects normalized channel values (substract means)
                                  transforms.RandomVerticalFlip(),
                                  transforms.RandomRotation(20),
                                  transforms.ToTensor()]),
            test_transform = transforms.Compose([transforms.ToPILImage(),
                                  transforms.Pad(64, padding_mode='reflect'), # 96 + 2*64 = 224
                                  transforms.ToTensor()]),
            in_memory = False,
            output_path = ".",
            #output_path = "drive/My Drive/dl_project_data/output",
            logger = {
                "api_token": "",
                "project_qualified_name": "elangenhan/hcd-experiments",
                "experiment_name": "ResNet152"
            }
)

https://ui.neptune.ai/elangenhan/hcd-experiments/e/HCDEX-62
Starting Training for <class 'architecture.ResNet152'> 
          Model-Params:
              Criterion:     <class 'torch.nn.modules.loss.BCEWithLogitsLoss'>
              Optimizer:     <class 'torch.optim.adam.Adam'>
              Learning Rate: 0.01
              Epochs:        2
              Batch size:    64
              
pretrained= False


  epoch    test_acc    test_f1    test_precision    test_recall    test_roc_auc    train_acc    train_f1    train_loss    train_precision    train_recall    train_roc_auc       dur
-------  ----------  ---------  ----------------  -------------  --------------  -----------  ----------  ------------  -----------------  --------------  ---------------  --------
      1      0.7697     0.6724            0.8043         0.5776          0.7401       0.7235      0.6197        0.6334             0.6890          0.5630           0.7798  396.4234


      2      0.7629     0.6208            0.8981         0.4743          0.7185       0.7721      0.7065        0.5732             0.7288          0.6855           0.8229  395.4406


Saving model...
Saving completed...
